In [6]:
type Estado = (Int, Int)   -- (litros en jarra A, litros en jarra B)
type Nodo = Estado
type Costo = Int

capA, capB :: Int
capA = 5
capB = 3

vecinos :: Estado -> [(Estado, Costo)]
vecinos (a,b) =
    [ ((capA, b), 1)                    -- Llenar jarra A
    , ((a, capB), 1)                    -- Llenar jarra B
    , ((0, b), 1)                       -- Vaciar jarra A
    , ((a, 0), 1)                       -- Vaciar jarra B
    , (transferirAB (a,b), 1)           -- Verter A → B
    , (transferirBA (a,b), 1)           -- Verter B → A
    ]

-- Verter de A hacia B
transferirAB :: Estado -> Estado
transferirAB (a,b) =
    let espacioB = capB - b
        cantidad = min a espacioB
    in (a - cantidad, b + cantidad)

-- Verter de B hacia A
transferirBA :: Estado -> Estado
transferirBA (a,b) =
    let espacioA = capA - a
        cantidad = min b espacioA
    in (a + cantidad, b - cantidad)

heuristica :: Estado -> Costo
heuristica (a,b) = abs (a - 4) + abs (b - 4)

-- Algoritmo A*
aStar :: (Nodo -> [(Nodo, Costo)]) -> (Nodo -> Costo) -> Nodo -> (Nodo -> Bool) -> [(Nodo, Costo)]
aStar sucesores heuristica inicio esMeta = buscar [(inicio, 0)] []
  where
    buscar [] _ = []
    buscar ((nodo, costo):cola) visitados
      | esMeta nodo = [(nodo, costo)]
      | nodo `elem` visitados = buscar cola visitados
      | otherwise =
          let nuevos = [(v, costo + c) | (v, c) <- sucesores nodo, v `notElem` visitados]
              ordenados = sortOn (\(v, c) -> c + heuristica v) (cola ++ nuevos)
          in (nodo, costo) : buscar ordenados (nodo : visitados)


meta :: Estado -> Bool
meta (a,b) = a == 4 || b == 4

inicio :: Estado
inicio = (0,0)

let resultado = aStar vecinos heuristica inicio meta
mapM_ print resultado

((0,0),0)
((5,0),1)
((5,3),2)
((2,3),2)
((0,3),1)
((3,0),2)
((3,3),3)
((5,1),4)
((2,0),3)
((0,2),4)
((5,2),5)
((4,3),6)